In [1]:
import os
import cv2 as cv
import numpy as np
import ipynb.fs.defs.Utils as Utils
import ipynb.fs.defs.FeatureMatching as FeatureMatching
import ipynb.fs.defs.GraphBuilding as GraphBuilding
import ipynb.fs.defs.BasicStitching as BasicStitching
import ipynb.fs.defs.SimpleGraphStitching as SimpleGraphStitching
import ipynb.fs.defs.MultiGraphStitching as MultiGraphStitching
import ipynb.fs.defs.MultiPatchStitching as MultiPatchStitching
import ipynb.fs.defs.Benchmark as Benchmark

In [2]:
#Directories
dataset_name = 'Helens' #Name of the dataset
dataset_dir = f'datasets/{dataset_name}' #Directory containing images of the dataset
stitching_dir = "stitched" #Directory to save stiched images
simple_output_dir ="output_simple"  #Directory for output of simple graph
average_output_dir ="output_average" #Directory for output of avg graph
multi_output_dir ="output_multi" #Directory for output of multi graph
results_dir = "results" #Directory for output results

In [3]:
#global variables
save_output = True #If True save in output all the computed intermediate results
save_images = True #If True save the images obtained in stitching procedure
verbose = True #If True visualize the results of stitching in the notebook
visualize = True #If True visualize benchmark at the end
noise_type = Utils.NoiseType.POINTS #Noise type used in the experiments
results = list() #List used to store results of different methods

In [4]:
#graph parameters
matching_threshold = 0.7 #Threshold used to compute matches
number_of_matches = 10 #Number of matches computed (multi-arc cardinality)
noise_std = 10 #Amount of noise
matches_th = 30 #Threshold used to compute good matches
RANSACmaxIters = 2000 #Maximum iterations for RANSAC (default 2000)

In [5]:
#general stitching parameters
idx_ref = None #Index of the reference image (None to use the automatically computed one)
idxs = None #Indexes of the images to stitch (None to use all the images of the dataset)
beautify = True #If True arrange the procedure to better visualize images
warp_shape = [10000,10000] #Shape of the stitched image

In [6]:
#This cell is needed to distinguish if the Main is running indipendently or in a ExperimentalResults run, the try block eventually 
#overwrites the parameters setting the ones coming from the experimental results, suppresing also all the visualizations
try:
    dataset_name = dataset_name_bench
    noise_type = noise_type_bench
    matching_threshold = matching_threshold_bench
    number_of_matches = number_of_matches_bench
    noise_std = noise_std_bench
    matches_th = matches_th_bench
    idx_ref = idx_ref_bench
    dataset_dir = f'datasets/{dataset_name}'
    save_output = False
    save_images = False
    verbose = False
    visualize = False
except NameError: _

In [7]:
#basic stitching parameters
basic_stitching_dir = "basic_stitching"
#simple graph stitching parameters
simple_graph_stitching_dir = "simple_graph"
#simple graph stitching parameters
edge_averaging_stitching_dir = "edge_averaging"
#multigraph stitching parameters
multi_graph_stitching_dir = "multi_graph"
#multipatch stitching parameters
multi_patch_stitching_dir = "multi_patch"
#edge averaging patch stitching parameters
average_patch_stitching_dir = "average_patch"

In [8]:
#Load images from the folder
imgs = Utils.load_images_from_folder(dataset_dir)

In [9]:
#compute normalization matrices
T_norm = Utils.get_normalization_matrix(imgs)

In [ ]:
#Compute a dict containing all the matches and the matrix containing the number of matches to be used as weights
matches_dict, weight_matrix = FeatureMatching.get_feature_matches(dataset_name,
            imgs,
            T_norm,
            matching_threshold = matching_threshold,
            number_of_matches = number_of_matches,
            noise_std = noise_std,
            matches_th = matches_th,
            RANSACmaxIters = RANSACmaxIters,
            save_output = save_output,
            save_images = save_images,
            output_dir = simple_output_dir,
            results_dir = results_dir,
            noise_type = noise_type,
            verbose = verbose
           )

In [ ]:
# Compute the matrices containing the homographies (M,Z) with other informations for synchronization and the adjacency matrix using
# a simple graph
M, Z, adj_matrix = GraphBuilding.build_simple_graph_matrices(dataset_name,
            imgs,
            matches_dict,
            output_dir = simple_output_dir,
            verbose = verbose,
            save_output = save_output
           )

# Compute the matrices containing the homographies (M,Z) with other informations for synchronization and the adjacency matrix using
# an edge-avering graph
M_average, Z_average, adj_matrix_average = GraphBuilding.build_edge_averaging_matrices(dataset_name, #Name of the dataset to be used
            imgs,
            matches_dict,
            output_dir = average_output_dir,
            verbose = verbose,
            save_output = save_output
           )
# Compute the matrices containing the homographies (M,Z) with other informations for synchronization and the adjacency matrix using
# a multi-graph
M_multi, Z_multi, C_multi, adj_matrix_multi = GraphBuilding.build_multi_graph_matrices(dataset_name, #Name of the dataset to be used
            imgs,
            matches_dict,
            output_dir = multi_output_dir,
            verbose = verbose,
            save_output = save_output
           )

In [ ]:
#If the index of the reference image is not set it is computed using adjacency matrix
if idx_ref is None:
    idx_ref = Utils.get_reference_node(adj_matrix)

In [ ]:
#Compute homographies and the image obtained from stitching using basic stitching
H_basic, basic_stitched_image = BasicStitching.basic_stitching(dataset_name,
                            imgs,
                            T_norm,
                            Z,
                            adj_matrix,
                            weight_matrix,
                            idx_ref = idx_ref,
                            verbose = verbose,
                            beautify = beautify,
                            save_output = save_output,
                            stitching_dir = stitching_dir,
                            basic_stitching_dir = basic_stitching_dir,
                            warp_shape = warp_shape
                            )
results.append({"name":"Basic Stitching","H":H_basic,"img":basic_stitched_image}) #Append the results into the list

In [ ]:
#Compute homographies and the image obtained from stitching using simple graph stitching
H_simple_graph, simple_graph_stitched_image, _ = SimpleGraphStitching.simple_graph_stitching(dataset_name,
                            imgs,
                            T_norm,
                            M,
                            idx_ref = idx_ref,
                            idxs = idxs,
                            verbose = verbose,
                            save_output = save_output,
                            beautify = beautify,
                            stitching_dir = stitching_dir,
                            graph_stitching_dir = simple_graph_stitching_dir,
                            warp_shape = warp_shape )
results.append({"name":"Simple Graph Stitching","H":H_simple_graph,"img":simple_graph_stitched_image}) #Append the results into the list

In [ ]:
#Compute homographies and the image obtained from stitching using multi graph stitching with matrices from edge averaging
H_average, edge_averaging_stitched_image, _ = SimpleGraphStitching.simple_graph_stitching(dataset_name,
                            imgs,
                            T_norm,
                            M_average, 
                            idx_ref = idx_ref,
                            idxs = idxs,
                            verbose = verbose,
                            save_output = save_output,
                            beautify = beautify,
                            stitching_dir = stitching_dir,
                            graph_stitching_dir = edge_averaging_stitching_dir,
                            warp_shape = warp_shape )
results.append({"name":"Edge Averaging Stitching","H":H_average,"img":edge_averaging_stitched_image}) #Append the results into the list

In [ ]:
#Compute homographies and the image obtained from stitching using multi graph stitching
H_multi_graph, multi_graph_stitched_image, _ = MultiGraphStitching.multi_graph_stitching(dataset_name,
                            imgs,
                            T_norm,
                            M_multi, 
                            C_multi,
                            idx_ref = idx_ref,
                            idxs = idxs,
                            verbose = verbose,
                            save_output = save_output,
                            beautify = beautify,
                            stitching_dir = stitching_dir,
                            graph_stitching_dir = multi_graph_stitching_dir,
                            warp_shape = warp_shape )
results.append({"name":"Multi-Graph Stitching","H":H_multi_graph,"img":multi_graph_stitched_image}) #Append the results into the list

In [ ]:
#Compute homographies and the image obtained from stitching using multi patch stitching
H_patch_graph, multi_patch_stitched_image = MultiPatchStitching.multi_patch_stitching(dataset_name,
                            imgs,
                            T_norm,
                            Z,
                            adj_matrix,
                            weight_matrix,
                            verbose = verbose,
                            idx_ref = idx_ref,
                            save_output = save_output,
                            beautify = beautify,
                            stitching_dir = stitching_dir,
                            graph_stitching_dir = multi_patch_stitching_dir,
                            warp_shape = warp_shape )
results.append({"name":"Multi-Patch Stitching","H":H_patch_graph,"img":multi_patch_stitched_image}) #Append the results into the list

In [ ]:
#Compute homographies and the image obtained from stitching using multi patch stitching with edge averaging
H_avg_patch_graph, average_patch_stitched_image = MultiPatchStitching.average_patch_stitching(dataset_name,
                            imgs,
                            T_norm,
                            Z,
                            adj_matrix,
                            weight_matrix,
                            verbose = verbose,
                            idx_ref = idx_ref,
                            save_output = save_output,
                            beautify = beautify,
                            stitching_dir = stitching_dir,
                            graph_stitching_dir = average_patch_stitching_dir,
                            warp_shape = warp_shape )
results.append({"name":"Multi-Patch Edge Averaging Stitching","H":H_avg_patch_graph,"img":average_patch_stitched_image}) #Append the results into the list

## Benchmark

In [ ]:
#The benchmark is run only if nosie is affecting the images, otherwise it will be meaningless
if noise_type != Utils.NoiseType.NO_NOISE:
    Benchmark.run_benchmark(dataset_name,
                                imgs,
                                T_norm = T_norm,
                                matching_threshold = matching_threshold,
                                matches_th = matches_th,
                                idx_ref = idx_ref,
                                results = results,
                                number_of_matches = number_of_matches,
                                noise_std = noise_std,
                                noise_type = noise_type,
                                visualize = visualize)